In [1]:
import numpy as np
import pandas as pd
import panel as pn
import joblib
import keras
import holoviews as hv
import hvplot.pandas
# from holoviews import opts
hv.extension('bokeh', 'matplotlib')

In [2]:
# Define file paths
file_paths = {
    "Sea_low": "inputs/Sea_low.csv",
    "Sea_high": "inputs/Sea_high.csv",
    "SJ": "inputs/SJ.csv",
    "IND": "inputs/IND.csv"
}

# Read CSV files directly into pandas DataFrames
Sea_l = pd.read_csv(file_paths["Sea_low"])
Sea_h = pd.read_csv(file_paths["Sea_high"])
SJ = pd.read_csv(file_paths["SJ"])
IND = pd.read_csv(file_paths["IND"])

def Ion_Simulator_PR(EC, Sacramento_X2, ion, WYT, month):
    # Convert Month name to numerical representation
    month_names = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
    month = month_names.index(month) + 1  # Convert month name to number

    results = {}

    for region in ["OMR", "SJRcorridor", "SouthDelta"]:
        # Define conditions for SEA, IND, and SJR
        if Sacramento_X2 < 81:
            sea_conditions = (
                (region == "OMR" and WYT in ["W", "AN"] and month in [1, 9, 10, 11, 12]) or
                (region == "OMR" and WYT in ["BN", "D"] and month in [1, 2, 8, 9, 10, 11, 12]) or
                (region == "OMR" and WYT == "C" and month in [1, 2, 6, 7, 8, 9, 10, 11, 12])
            )
            ind_conditions = (
                (region == "SJRcorridor" and WYT == "D" and month in [6, 7, 8, 9, 10, 11]) or
                (region == "SJRcorridor" and WYT == "C" and month in [6, 7, 8, 9, 10, 11]) or
                (region == "SouthDelta" and WYT in ["AN", "BN", "D"] and month in [1, 2, 8, 9, 10, 11, 12]) or
                (region == "SouthDelta" and WYT == "C" and month in [1, 2, 6, 7, 8, 9, 10, 11, 12])
            )
        else:
            sea_conditions = (
                (region == "OMR" and WYT in ["W", "AN"] and month in [1, 9, 10, 11, 12]) or
                (region == "OMR" and WYT in ["BN", "D", "C"] and month in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
            )
            ind_conditions = (
                (region == "SouthDelta" and WYT in ["BN", "D"] and month in [1, 2, 8, 9, 10, 11, 12]) or
                (region == "SouthDelta" and WYT == "C" and month in [1, 2, 6, 7, 8, 9, 10, 11, 12])
            )

        # Select coefficients based on conditions
        if sea_conditions:
            coeffs = Sea_l if EC < 250 else Sea_h
        elif ind_conditions:
            coeffs = IND
        else:
            coeffs = SJ

        # Select the row for the specific Ion
        coeffs_row = coeffs[coeffs['ion'] == ion]

        # Calculate Ion level based on selected coefficients
        if sea_conditions:  # Use the complex formula for SEA and IND
            simulated = coeffs_row['K1'] + coeffs_row['K2']*np.sqrt(EC) + coeffs_row['K3']*EC + coeffs_row['K4']*EC**1.5 + coeffs_row['K5']*EC**2 + coeffs_row['K6']*EC**2.5
        else:  # Use the simpler formula for SJR
            simulated = coeffs_row['a']*EC**2 + coeffs_row['b']*EC + coeffs_row['c']

        results[region] = simulated.iloc[0]  # Assuming only one row matches

    return results


def Ion_Simulator(EC, Sacramento_X2, Ion, WYT, month):
    regions = ["OMR", "SJRcorridor", "SouthDelta"]  # List of regions to iterate over
    results = []  # Initialize an empty list to store results

    for Region in regions:
        X = pd.DataFrame(columns=['EC', 'Sacramento X2', 'AN', 'BN', 'C', 'D', 'W', 'OMR', 'SJRcorridor', 'SouthDelta', 'April', 'August', 'December', 'February', 'January', 'July', 'June', 'March', 'May', 'November', 'October', 'September'])

        X.loc[0, "EC"] = (EC - 50) / (3500 - 50)
        X.loc[0, 'Sacramento X2'] = Sacramento_X2 / 100
        X.loc[0, 'AN'] = 1 if WYT == "AN" else 0
        X.loc[0, 'BN'] = 1 if WYT == "BN" else 0
        X.loc[0, 'C'] = 1 if WYT == "C" else 0
        X.loc[0, 'D'] = 1 if WYT == "D" else 0
        X.loc[0, 'W'] = 1 if WYT == "W" else 0

        X.loc[0, 'OMR'] = 1 if Region == "OMR" else 0
        X.loc[0, 'SJRcorridor'] = 1 if Region == "SJRcorridor" else 0
        X.loc[0, 'SouthDelta'] = 1 if Region == "SouthDelta" else 0

        X.loc[0,'April']=1 if month=="April" else 0
        X.loc[0,'August']=1 if month=="August" else 0
        X.loc[0,'December']=1 if month=="December" else 0
        X.loc[0,'February']=1 if month=="February" else 0
        X.loc[0,'January']=1 if month=="January" else 0
        X.loc[0,'July']=1 if month=="July" else 0
        X.loc[0,'June']=1 if month=="June" else 0
        X.loc[0,'March']=1 if month=="March" else 0
        X.loc[0,'May']=1 if month=="May" else 0
        X.loc[0,'November']=1 if month=="November" else 0
        X.loc[0,'October']=1 if month=="October" else 0
        X.loc[0,'September']=1 if month=="September" else 0

        X1 = X.astype(float)

        # Load models
        ANN_final = keras.models.load_model(f'Models/ANN_{Ion}.h5')
        # MLR_final = joblib.load(filename=f'Models/MLR_{Ion}.pkl')
        RT_final = joblib.load(filename=f'Models/RT_{Ion}.pkl')
        RF_final = joblib.load(filename=f'Models/RF_{Ion}.pkl')
        GB_final = joblib.load(filename=f'Models/GB_{Ion}.pkl')

        df = pd.DataFrame(columns=['Region', 'RT', 'GB', 'RF', 'ANN'])
        df.loc[0, 'Region'] = Region
        df.loc[0, 'RT'] = round(RT_final.predict(X1)[0], 2)
        df.loc[0, 'GB'] = round(GB_final.predict(X1)[0], 2)
        df.loc[0, 'RF'] = round(RF_final.predict(X1)[0], 2)
        df.loc[0, 'ANN'] = round(ANN_final.predict(X1)[0][0].astype(float), 2)

        results.append(df)

    # Concatenate all results into a single DataFrame
    final_results = pd.concat(results, ignore_index=True)
    return final_results

def combined_Ion_Simulation(EC, Sacramento_X2, Ion, WYT, month):
    # First, run the machine learning simulation
    ml_results = Ion_Simulator(EC, Sacramento_X2, Ion, WYT, month)
    
    # Then, run the TT method simulation
    pr_results = Ion_Simulator_PR(EC, Sacramento_X2, Ion, WYT, month)
    
    # Initialize a list to hold all rows before creating the DataFrame
    combined_data = []
    
    # Iterate through the machine learning results to combine with TT
    for index, row in ml_results.iterrows():
        region = row['Region']
        combined_data.append({
            'Region': region,
            'RT': row['RT'],
            'GB': row['GB'],
            'RF': row['RF'],
            'ANN': row['ANN'],
            'PR': pr_results[region]  # Add the TT result for the corresponding region
        })
    
    # Convert the list of dictionaries to a DataFrame
    combined_results = pd.DataFrame(combined_data)
    return combined_results


In [3]:
# Initialize the Panel extension
pn.extension()
custom_style1 = {
    'background': '#f9f9f9',
    'border': '1px solid black',
    'padding': '10px',
    'box-shadow': '5px 6px 7px #bcbcbc'
}
# Dictionary of ion thresholds
ion_thresholds = {
    'Cl': 250,
    'SO4': 250,
    'Na': 60,
    'TDS': 500,
}

# Define logo
logo = pn.pane.PNG('Logo.png', width=551, height=91, sizing_mode='fixed')

fig1_title = pn.pane.Markdown("### Study Area", width=605, align='center')
spacer_left = pn.Spacer(width=250)  # Adjust width as needed
spacer_right = pn.Spacer(width=250)  # Adjust width as needed
fig1 = pn.pane.PNG('Fig1.png', width=605, height=753, sizing_mode='fixed')  
fig1_with_title = pn.Column(pn.Row(spacer_left, fig1_title, spacer_right), fig1)

# input widgets for the dashboard
Ion = pn.widgets.Select(name='Ion', options=['Alkalinity','Br','Ca', 'Cl', 'K', 'Mg','Na', 'SO4','TDS'], value='Br',styles = custom_style1)
EC = pn.widgets.FloatSlider(name='EC', start=100, end=3500, value=200,styles = custom_style1)

# EC = pn.widgets.FloatInput(name='EC', start=100, end=3500, value=200.0, styles=custom_style1)
Sacramento_X2 = pn.widgets.FloatSlider(name='Sacramento_X2', start=0, end=100, value=90,styles = custom_style1)
# Sacramento_X2 = pn.widgets.FloatInput(name='Sacramento X2', start=0, end=100, value=90.0, styles=custom_style1)
WYT = pn.widgets.Select(name='Water Year Type (WYT)', options=['AN', 'BN', 'C', 'D', 'W'], value='AN',styles = custom_style1)
Region = pn.widgets.Select(name='Sub-Region', options=['OMR', 'SJRcorridor', 'SouthDelta'], value='OMR',styles = custom_style1)
month = pn.widgets.Select(name='Month', options=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], value='September',styles = custom_style1)

# function to compute the ion simulator and generate the output plot
def compute_ion_simulator(event):
    # Call the combined Ion Simulation function to get results for all regions
    result = combined_Ion_Simulation(EC.value, Sacramento_X2.value, Ion.value, WYT.value, month.value)
    
    # Melt the result for plotting
    melted_result = result.melt(id_vars=['Region'], var_name='Model', value_name='Value')
    
    # Plotting
    ion_name = Ion.value
    bar_chart = melted_result.hvplot.bar(x='Model', y='Value', by='Region', 
                                         xlabel='Prediction Models', ylabel=f'{ion_name} [mg/L]',
                                         hover=True, legend='top_right', width=800, height=500, rot=45,ylim=(0, None))

    # Check if the selected ion has a threshold and add a horizontal line
    if ion_name in ion_thresholds:
        threshold = ion_thresholds[ion_name]
        threshold_line = hv.HLine(threshold).opts(color='red', line_dash='dashed', line_width=2.0)
        # Overlay the threshold line on the bar chart
        bar_chart = bar_chart * threshold_line
    
    output_plot.object = bar_chart

# The output plot object
output_plot = pn.pane.HoloViews()

# Markdown pane for the top part of the dashboard
instructions_top = pn.pane.Markdown("""
## Ion Simulator Dashboard

This dashboard allows you to simulate ion concentrations based on easyly availble parameters. 
    Use the sliders and dropdown menus to select the desired values for EC, Sacramento_X2, Ion, WYT, 
    and Month. Then click the 'Compute' button to generate a bar chart of the predicted ion concentrations in three sub-regions.
### Instructions:

1. Adjust the sliders and drop-down menus to select the desired input values.
2. Click the **Compute** button to run the simulation.
3. The bar chart will display the predicted ion concentrations for different machine learning models and parametric regression method (prepared by Tetra Tech company).

### Notes:

- Electrical conductivity (EC) is measured in microsiemens per centimeter (µS/cm).
- Sacramento_X2 is the percentage of Sacramento River flow that is estimated to reach the Delta.
The exact location of the Sacramento X2 point is determined by the California Department of Water Resources (DWR)
 based on the specific hydraulic conditions and water flows in the Sacramento River. 
The DWR uses a combination of hydrological models, flow measurements, and other data to determine the location of the Sacramento X2 point.
- The Water Year Type (WYT) is a classification of the water year based on its hydrological characteristics.
Water Year Type that includes the following categories: 1- Wet (W), 2- Critical (C), 3- Dry (D), 4- Above-Normal (AN), 5- Below-Normal (BN)
- Region refers to monitoring regions that includes: 1- Old-Middle River (OMR), 2- San Joaquin River Corridor (SJRcorridor), and 3- South Delta (SouthDelta).
- Month refers to the month of the year.
- Prediction models: Regression Trees: RT, Gradient Boosting: GB, Random Forest: RF, Artificial Neural Networks: ANN, Parametric Regression method prepared by Tetra Tech: PR
                                    
- **<span style='color: red;'>The red dashed line</span>** serves as an indicator of the acceptable threshold level for ion concentration in water, 
                                    beyond which the water quality may not meet standards for human consumption or use, based on guidelines from the
                                     EPA and the World Health Organization (WHO); {Cl:250, SO4:250, Na:60, TDS:500}
Sources of this information include: 
- EPA: [National Primary Drinking Water Regulations](https://www.epa.gov/ground-water-and-drinking-water/national-primary-drinking-water-regulations)
- World Health Organization (WHO): [Guidelines for Drinking-water Quality, 4th edition](https://www.who.int/publications/i/item/9789241549950)

""")

# Markdown pane for the bottom part of the dashboard
instructions_bottom = pn.pane.Markdown("""
                                       
### References:
- Namadi, P., He, M. & Sandhu, P. Salinity-constituent conversion in South Sacramento-San Joaquin Delta of California via machine learning. 
                                       Earth Sci Inform 15, 1749–1764 (2022). [https://doi.org/10.1007/s12145-022-00828-1](https://doi.org/10.1007/s12145-022-00828-1)

- Namadi, P., He, M. & Sandhu, P. Modeling ion constituents in the Sacramento-San Joaquin Delta using multiple machine learning approaches. 
                                       Journal of Hydroinformatics (2023). [https://doi.org/10.2166/hydro.2023.158](https://doi.org/10.2166/hydro.2023.158)

- Paul Hutton, Arushi Sinha, and Sujoy Roy. 2022. Simplified Approach for Estimating Salinity Constituent Concentrations in the San Francisco Estuary & Sacramento-San Joaquin River Delta. 
                                       [Report's link](https://rtdf.info/public_docs/Miscellaneous%20RTDF%20Web%20Page%20Information/Other%20MWQP%20and%20DWR%20Publications/2022-07-21%20MWQI%20Conservative%20Constituents%20User%20Guide_formatted.pdf)

                             
### Disclaimer: this dashboard is still in beta.

Thank you for evaluating the Ion Simulator Dashboard.  

If you have feedback, suggestions or questions, please contact Peyman Namadi or Kevin He (Peyman.Hosseinzadehnamadi@Water.ca.gov  or  Kevin.He@Water.ca.gov)
""")


compute_button = pn.widgets.Button(name='Compute', button_type='primary')
compute_button.on_click(compute_ion_simulator)

dashboard_content = pn.Column(
    instructions_top,
    pn.Row(Ion),
    pn.Row(EC, Sacramento_X2),
    pn.Row(WYT, month),  
    output_plot,
    compute_button,
    instructions_bottom
)


layout = pn.Row(
    dashboard_content,  
    fig1_with_title  
)


dashboard = pn.Column(logo, layout)

# Display the dashboard
#dashboard.show()
dashboard.servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'b4215c3b-062a-48a9-815f-afcb2ea38fba': {'version…

1/1 [==============================] - 0s 81ms/step
